In [1]:
pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

# Cargar los datos
df = pd.read_excel(r"data/empleabilidad.xlsx")

# Crear columna de Quimestre personalizada
def asignar_quimestre(mes):
    if mes == 2:
        return 'Q1'
    elif mes == 5:
        return 'Q2'
    elif mes == 9:
        return 'Q3'
    elif mes == 11:
        return 'Q4'
    else:
        return None

df['Quimestre'] = df['Mes.1'].apply(asignar_quimestre)

# Crear columna para determinar si está empleado
df['Esta_empleado'] = df['SALARIO.1'].notnull() | df['RUCEMP.1'].notnull()

# Agrupar por año y quimestre
empleabilidad = df.groupby(['Anio.1', 'Quimestre']).agg(
    total_graduados=('IdentificacionBanner.1', 'nunique'),
    empleados=('Esta_empleado', 'sum'),
    salario_promedio=('SALARIO.1', 'mean')
).reset_index()

# Calcular tasa de empleabilidad
empleabilidad['tasa_empleabilidad'] = empleabilidad['empleados'] / empleabilidad['total_graduados']

# Mostrar resultados
print(empleabilidad)


   Anio.1 Quimestre  total_graduados  empleados  salario_promedio  \
0    2024        Q1            36059      23994       1136.900481   
1    2024        Q2            36059      23974       1147.771761   
2    2024        Q3            36059      23818       1162.583545   
3    2024        Q4            36059      24085       1161.393170   

   tasa_empleabilidad  
0            0.665409  
1            0.664855  
2            0.660529  
3            0.667933  


In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Leer el archivo
df = pd.read_excel(r"data/empleabilidad.xlsx")

# Asignar quimestre
def asignar_quimestre(mes):
    if mes == 2:
        return 'Q1'
    elif mes == 5:
        return 'Q2'
    elif mes == 9:
        return 'Q3'
    elif mes == 11:
        return 'Q4'
    else:
        return None

df['Quimestre'] = df['Mes.1'].apply(asignar_quimestre)
df['Esta_empleado'] = df['SALARIO.1'].notnull() | df['RUCEMP.1'].notnull()
df = df[df['Quimestre'].notnull()]

# Crear campo Periodo (año-quimestre)
df['Periodo'] = df['Anio.1'].astype(str) + ' ' + df['Quimestre']

# App Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Tasa de Empleabilidad por Quimestre"),

    html.Label("Facultad:"),
    dcc.Dropdown(
        id='facultad-filter',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        value=None,
        placeholder='Selecciona una facultad'
    ),

    html.Label("Carrera:"),
    dcc.Dropdown(
        id='carrera-filter',
        options=[],
        value=None,
        placeholder='Selecciona una carrera'
    ),

    dcc.Graph(id='empleabilidad-grafico')
])

@app.callback(
    Output('carrera-filter', 'options'),
    Input('facultad-filter', 'value')
)
def update_carreras(facultad):
    if facultad is None:
        return []
    carreras = df[df['FACULTAD'] == facultad]['CarreraHomologada.1'].dropna().unique()
    return [{'label': c, 'value': c} for c in sorted(carreras)]

@app.callback(
    Output('empleabilidad-grafico', 'figure'),
    Input('facultad-filter', 'value'),
    Input('carrera-filter', 'value')
)
def update_graph(facultad, carrera):
    filtered = df.copy()
    if facultad:
        filtered = filtered[filtered['FACULTAD'] == facultad]
    if carrera:
        filtered = filtered[filtered['CarreraHomologada.1'] == carrera]

    # Agrupación
    resumen = filtered.groupby(['Periodo']).agg(
        empleados=('Esta_empleado', 'sum'),
        total=('IdentificacionBanner.1', 'nunique')
    ).reset_index()
    resumen['tasa_empleabilidad'] = resumen['empleados'] / resumen['total']

    fig = px.line(
        resumen,
        x='Periodo',
        y='tasa_empleabilidad',
        title='Tasa de Empleabilidad',
        markers=True,
        labels={'tasa_empleabilidad': 'Tasa de empleo'}
    )
    fig.update_layout(xaxis_tickangle=-45)
    return fig

if __name__ == '__main__':
    app.run(debug=True)


SALARIO

In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Leer archivo
df = pd.read_excel(r"data/empleabilidad.xlsx")

# Preparar columnas necesarias
def asignar_quimestre(mes):
    if mes == 2: return 'Q1'
    elif mes == 5: return 'Q2'
    elif mes == 9: return 'Q3'
    elif mes == 11: return 'Q4'
    else: return None

df['Quimestre'] = df['Mes.1'].apply(asignar_quimestre)
df['Periodo'] = df['Anio.1'].astype(str) + ' ' + df['Quimestre']
df = df[df['Quimestre'].notnull()]
df = df[df['SALARIO.1'].notnull()]

# Crear aplicación Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Distribución de Salarios por Periodo"),

    html.Label("Facultad:"),
    dcc.Dropdown(
        id='facultad-salario',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        placeholder="Selecciona una facultad"
    ),

    html.Label("Carrera:"),
    dcc.Dropdown(id='carrera-salario', placeholder="Selecciona una carrera"),

    dcc.Graph(id='boxplot-salario'),

    html.H4("Resumen estadístico del salario"),
    html.Div(id='resumen-estadistico')
])

@app.callback(
    Output('carrera-salario', 'options'),
    Input('facultad-salario', 'value')
)
def actualizar_carreras(facultad):
    if facultad:
        carreras = df[df['FACULTAD'] == facultad]['CarreraHomologada.1'].dropna().unique()
        return [{'label': c, 'value': c} for c in sorted(carreras)]
    return []

@app.callback(
    Output('boxplot-salario', 'figure'),
    Output('resumen-estadistico', 'children'),
    Input('facultad-salario', 'value'),
    Input('carrera-salario', 'value')
)
def actualizar_boxplot_y_resumen(facultad, carrera):
    filtered = df.copy()
    if facultad:
        filtered = filtered[filtered['FACULTAD'] == facultad]
    if carrera:
        filtered = filtered[filtered['CarreraHomologada.1'] == carrera]

    # BOXPLOT
    fig = px.box(
        filtered,
        x='Periodo',
        y='SALARIO.1',
        title='Distribución de Salarios por Periodo',
        labels={'SALARIO.1': 'Salario', 'Periodo': 'Año-Quimestre'},
        points='outliers'  # Solo muestra puntos fuera del rango normal
    )
    fig.update_layout(xaxis_tickangle=-45)

    # RESUMEN ESTADÍSTICO
    resumen = filtered['SALARIO.1'].describe()
    texto_resumen = html.Ul([
        html.Li(f"Media: ${resumen['mean']:.2f}"),
        html.Li(f"Mediana: ${filtered['SALARIO.1'].median():.2f}"),
        html.Li(f"Desviación estándar: ${resumen['std']:.2f}"),
        html.Li(f"Q1 (percentil 25): ${resumen['25%']:.2f}"),
        html.Li(f"Q3 (percentil 75): ${resumen['75%']:.2f}"),
        html.Li(f"Máximo: ${resumen['max']:.2f}"),
        html.Li(f"Mínimo: ${resumen['min']:.2f}"),
        html.Li(f"Número de observaciones: {int(resumen['count'])}")
    ])

    return fig, texto_resumen

if __name__ == '__main__':
    app.run(debug=True)


SECTOR ECONOMICO

In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Leer archivo
df = pd.read_excel(r"data/empleabilidad.xlsx")

# Preprocesamiento
df['Esta_empleado'] = df['SALARIO.1'].notnull() | df['RUCEMP.1'].notnull()
df = df[df['SECTOR'].notnull()]  # Asegurar que SECTOR no esté vacío

# App
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Distribución de Graduados por Sector Económico"),

    html.Label("Facultad:"),
    dcc.Dropdown(
        id='facultad-sector',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        placeholder="Selecciona una facultad"
    ),

    html.Label("Carrera:"),
    dcc.Dropdown(id='carrera-sector', placeholder="Selecciona una carrera"),

    dcc.Graph(id='grafico-sector')
])

@app.callback(
    Output('carrera-sector', 'options'),
    Input('facultad-sector', 'value')
)
def actualizar_carreras_sector(facultad):
    if facultad:
        carreras = df[df['FACULTAD'] == facultad]['CarreraHomologada.1'].dropna().unique()
        return [{'label': c, 'value': c} for c in sorted(carreras)]
    return []

@app.callback(
    Output('grafico-sector', 'figure'),
    Input('facultad-sector', 'value'),
    Input('carrera-sector', 'value')
)
def actualizar_grafico_sector(facultad, carrera):
    filtered = df[df['Esta_empleado'] == True].copy()
    if facultad:
        filtered = filtered[filtered['FACULTAD'] == facultad]
    if carrera:
        filtered = filtered[filtered['CarreraHomologada.1'] == carrera]

    conteo_sector = filtered['SECTOR'].value_counts().reset_index()
    conteo_sector.columns = ['Sector Económico', 'Número de Graduados']

    fig = px.bar(
        conteo_sector,
        x='Número de Graduados',
        y='Sector Económico',
        orientation='h',
        title='Distribución por Sector Económico',
        labels={'Número de Graduados': 'Cantidad'}
    )
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})

    return fig

if __name__ == '__main__':
    app.run(debug=True)


Tamaño de empresa

In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Leer archivo
df = pd.read_excel(r"data/empleabilidad.xlsx")

# Crear campo de "empleado"
df['Esta_empleado'] = df['SALARIO.1'].notnull() | df['RUCEMP.1'].notnull()
df = df[df['Cantidad de empleados'].notnull()]

# Clasificar tamaño de empresa
def clasificar_tamano(n):
    if n <= 10:
        return 'Microempresa (1–10)'
    elif n <= 50:
        return 'Pequeña (11–50)'
    elif n <= 200:
        return 'Mediana (51–200)'
    else:
        return 'Grande (200+)'

df['Tamaño Empresa'] = df['Cantidad de empleados'].apply(clasificar_tamano)

# App Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Distribución por Tamaño de Empresa"),

    html.Label("Facultad:"),
    dcc.Dropdown(
        id='facultad-tamano',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        placeholder="Selecciona una facultad"
    ),

    html.Label("Carrera:"),
    dcc.Dropdown(id='carrera-tamano', placeholder="Selecciona una carrera"),

    dcc.Graph(id='grafico-tamano-empresa')
])

@app.callback(
    Output('carrera-tamano', 'options'),
    Input('facultad-tamano', 'value')
)
def actualizar_carreras(facultad):
    if facultad:
        carreras = df[df['FACULTAD'] == facultad]['CarreraHomologada.1'].dropna().unique()
        return [{'label': c, 'value': c} for c in sorted(carreras)]
    return []

@app.callback(
    Output('grafico-tamano-empresa', 'figure'),
    Input('facultad-tamano', 'value'),
    Input('carrera-tamano', 'value')
)
def actualizar_grafico_tamano(facultad, carrera):
    filtered = df[df['Esta_empleado'] == True].copy()
    if facultad:
        filtered = filtered[filtered['FACULTAD'] == facultad]
    if carrera:
        filtered = filtered[filtered['CarreraHomologada.1'] == carrera]

    conteo_tamano = filtered['Tamaño Empresa'].value_counts().reset_index()
    conteo_tamano.columns = ['Tamaño Empresa', 'Número de Graduados']

    fig = px.bar(
        conteo_tamano,
        x='Tamaño Empresa',
        y='Número de Graduados',
        title='Distribución de Graduados por Tamaño de Empresa',
        labels={'Tamaño Empresa': 'Tamaño de empresa'},
        color='Tamaño Empresa'
    )
    fig.update_layout(xaxis={'categoryorder': 'array',
                             'categoryarray': ['Microempresa (1–10)', 'Pequeña (11–50)', 'Mediana (51–200)', 'Grande (200+)']})
    return fig

if __name__ == '__main__':
    app.run(debug=True)


Ranking de carreras por empleabilidad

In [6]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Leer archivo
df = pd.read_excel(r"data/empleabilidad.xlsx")

# Preparar datos
df['Esta_empleado'] = df['SALARIO.1'].notnull() | df['RUCEMP.1'].notnull()

def asignar_quimestre(mes):
    if mes == 2: return 'Q1'
    elif mes == 5: return 'Q2'
    elif mes == 9: return 'Q3'
    elif mes == 11: return 'Q4'
    else: return None

df['Quimestre'] = df['Mes.1'].apply(asignar_quimestre)
df['Periodo'] = df['Anio.1'].astype(str) + ' ' + df['Quimestre']
df = df[df['Quimestre'].notnull()]

# App Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Ranking de Carreras por Empleabilidad"),

    html.Label("Filtrar por facultad (opcional):"),
    dcc.Dropdown(
        id='facultad-ranking',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        placeholder="Selecciona una facultad"
    ),

    html.Label("Filtrar por periodo (opcional):"),
    dcc.Dropdown(
        id='periodo-ranking',
        options=[{'label': p, 'value': p} for p in sorted(df['Periodo'].dropna().unique())],
        placeholder="Selecciona un periodo"
    ),

    dcc.Graph(id='grafico-ranking')
])

@app.callback(
    Output('grafico-ranking', 'figure'),
    Input('facultad-ranking', 'value'),
    Input('periodo-ranking', 'value')
)
def actualizar_ranking(facultad, periodo):
    filtered = df.copy()
    if facultad:
        filtered = filtered[filtered['FACULTAD'] == facultad]
    if periodo:
        filtered = filtered[filtered['Periodo'] == periodo]

    resumen = filtered.groupby('CarreraHomologada.1').agg(
        empleados=('Esta_empleado', 'sum'),
        total=('IdentificacionBanner.1', 'nunique')
    ).reset_index()

    resumen['Tasa de Empleabilidad'] = resumen['empleados'] / resumen['total']
    resumen = resumen[resumen['total'] > 0]  # Evitar divisiones por cero
    resumen = resumen.sort_values('Tasa de Empleabilidad', ascending=False)

    fig = px.bar(
        resumen,
        x='CarreraHomologada.1',
        y='Tasa de Empleabilidad',
        title='Ranking de Carreras por Empleabilidad',
        labels={'CarreraHomologada.1': 'Carrera'},
        color='Tasa de Empleabilidad'
    )
    fig.update_layout(xaxis_tickangle=-45)
    return fig

if __name__ == '__main__':
    app.run(debug=True)


CARRERAS CRITICAS

In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output, dash_table
from dash.dash_table.Format import Format, Scheme
from sklearn.linear_model import LinearRegression
import numpy as np

# === Leer archivo
df = pd.read_excel(r"data/empleabilidad.xlsx")

# === Preprocesamiento ===
df['Esta_empleado'] = df['SALARIO.1'].notnull() | df['RUCEMP.1'].notnull()

def asignar_quimestre(mes):
    if mes == 2: return 'Q1'
    elif mes == 5: return 'Q2'
    elif mes == 9: return 'Q3'
    elif mes == 11: return 'Q4'
    else: return None

df['Quimestre'] = df['Mes.1'].apply(asignar_quimestre)
df['Periodo'] = df['Anio.1'].astype(str) + ' ' + df['Quimestre']
df = df[df['Quimestre'].notnull()]

# === App Dash ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("⚠️ Carreras Críticas por Baja Empleabilidad"),

    html.Label("Umbral de alerta (% de empleabilidad mínima):"),
    dcc.Slider(
        id='umbral-slider',
        min=0.1, max=0.9, step=0.05, value=0.6,
        marks={i / 10: f'{int(i * 10)}%' for i in range(1, 10)}
    ),

    html.Label("Filtrar por facultad (opcional):"),
    dcc.Dropdown(
        id='facultad-alerta',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        placeholder="Selecciona una facultad"
    ),

    dash_table.DataTable(
        id='tabla-alertas',
        columns=[
            {"name": "Carrera", "id": "Carrera"},
            {"name": "Mínima tasa quimestral", "id": "MinTasa", "type": "numeric",
             "format": Format(precision=1, scheme=Scheme.percentage)},
            {"name": "Pendiente (tendencia)", "id": "Pendiente", "type": "numeric",
             "format": Format(precision=2, scheme=Scheme.fixed)},
            {"name": "Tipo de alerta", "id": "Tipo"}
        ],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_data_conditional=[
            {
                'if': {'filter_query': '{MinTasa} < 0.6', 'column_id': 'MinTasa'},
                'backgroundColor': '#ffe6e6'
            },
            {
                'if': {'filter_query': '{Pendiente} < 0', 'column_id': 'Pendiente'},
                'backgroundColor': '#fff0cc'
            }
        ]
    )
])

# === Callback principal ===
@app.callback(
    Output('tabla-alertas', 'data'),
    Input('umbral-slider', 'value'),
    Input('facultad-alerta', 'value')
)
def detectar_alertas(umbral, facultad):
    filtered = df.copy()
    if facultad:
        filtered = filtered[filtered['FACULTAD'] == facultad]

    resumen = filtered.groupby(['CarreraHomologada.1', 'Periodo']).agg(
        empleados=('Esta_empleado', 'sum'),
        total=('IdentificacionBanner.1', 'nunique')
    ).reset_index()
    resumen['tasa'] = resumen['empleados'] / resumen['total']
    resumen = resumen[resumen['total'] >= 1]

    carreras = []

    for carrera, grupo in resumen.groupby('CarreraHomologada.1'):
        grupo = grupo.sort_values('Periodo')
        tasas = grupo['tasa'].values

        min_tasa = tasas.min()
        if len(grupo) >= 2:
            X = np.arange(len(grupo)).reshape(-1, 1)
            y = tasas
            modelo = LinearRegression().fit(X, y)
            pendiente = modelo.coef_[0]
        else:
            pendiente = 0

        alerta_tasa = min_tasa < umbral
        alerta_trend = pendiente < 0

        if alerta_tasa or alerta_trend:
            if alerta_tasa and alerta_trend:
                tipo = "Ambas"
            elif alerta_tasa:
                tipo = "Tasa baja"
            else:
                tipo = "Tendencia descendente"

            carreras.append({
                "Carrera": carrera,
                "MinTasa": min_tasa,
                "Pendiente": pendiente,
                "Tipo": tipo
            })

    if not carreras:
        return [{
            "Carrera": "⚠ No se encontraron carreras críticas con los filtros actuales",
            "MinTasa": None,
            "Pendiente": None,
            "Tipo": ""
        }]

    return carreras

# === Run ===
if __name__ == '__main__':
    app.run(debug=True)


Comparacion por cohortes

In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# === Leer los datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx")

# === Determinar si está empleado (empleo de cualquier tipo) ===
df['Esta_empleado'] = df['SALARIO.1'].notnull() | df['RUCEMP.1'].notnull()

# === Crear la app Dash ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("📊 Comparación de Cohortes por Empleabilidad"),

    html.Label("Selecciona facultad:"),
    dcc.Dropdown(
        id='facultad',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        placeholder="Selecciona una facultad"
    ),

    html.Label("Selecciona carrera:"),
    dcc.Dropdown(id='carrera'),

    html.Label("¿Solo empleo formal?"),
    dcc.Checklist(
        id='solo-formal',
        options=[{'label': 'Sí', 'value': 'formal'}],
        value=[]
    ),

    html.Label("Tipo de gráfico:"),
    dcc.RadioItems(
        id='tipo-grafico',
        options=[
            {'label': 'Líneas', 'value': 'line'},
            {'label': 'Barras', 'value': 'bar'}
        ],
        value='line',
        labelStyle={'display': 'inline-block', 'margin-right': '15px'}
    ),

    dcc.Graph(id='grafico-cohortes')
])

# === Callback: actualizar lista de carreras ===
@app.callback(
    Output('carrera', 'options'),
    Input('facultad', 'value')
)
def actualizar_carreras(facultad):
    if facultad:
        carreras = df[df['FACULTAD'] == facultad]['CarreraHomologada.1'].dropna().unique()
        return [{'label': c, 'value': c} for c in sorted(carreras)]
    return []

# === Callback: generar gráfico ===
@app.callback(
    Output('grafico-cohortes', 'figure'),
    Input('facultad', 'value'),
    Input('carrera', 'value'),
    Input('solo-formal', 'value'),
    Input('tipo-grafico', 'value')
)
def graficar_cohortes(facultad, carrera, solo_formal, tipo_grafico):
    data = df.copy()

    if facultad:
        data = data[data['FACULTAD'] == facultad]
    if carrera:
        data = data[data['CarreraHomologada.1'] == carrera]

    # Total: todos los graduados por cohorte
    total = data.groupby('AnioGraduacion.1')['IdentificacionBanner.1'].nunique()

    # Empleados: según filtro
    if 'formal' in solo_formal:
        empleados_df = data[data['Empleo formal'].astype(str).str.upper() == 'EMPLEO FORMAL']
    else:
        empleados_df = data[data['Esta_empleado']]

    empleados = empleados_df.groupby('AnioGraduacion.1')['IdentificacionBanner.1'].nunique()

    # Juntar datos
    resumen = pd.DataFrame({'empleados': empleados, 'total': total}).reset_index()
    resumen = resumen[resumen['total'] > 0]
    resumen['tasa'] = resumen['empleados'] / resumen['total']
    resumen = resumen.sort_values('AnioGraduacion.1')

    titulo = f'Tasa de empleabilidad por cohorte {"(solo empleo formal)" if "formal" in solo_formal else ""}'

    # === Crear gráfico ===
    if tipo_grafico == 'bar':
        fig = px.bar(resumen, x='AnioGraduacion.1', y='tasa',
                     labels={'AnioGraduacion.1': 'Año de graduación', 'tasa': 'Tasa de empleo'},
                     title=titulo, text_auto='.1%')
    else:
        fig = px.line(resumen, x='AnioGraduacion.1', y='tasa',
                      labels={'AnioGraduacion.1': 'Año de graduación', 'tasa': 'Tasa de empleo'},
                      title=titulo, markers=True)
        fig.update_traces(mode='lines+markers')
        fig.update_yaxes(tickformat=".0%")

    return fig

# === Ejecutar app ===
if __name__ == '__main__':
    app.run(debug=True)


Riesgo de desempleo en el tiempo

In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# === Leer datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx")

# === Marcar si está empleado ===
df['Esta_empleado'] = df['SALARIO.1'].notnull() | df['RUCEMP.1'].notnull()

# === App Dash ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("📉 Riesgo de Desempleo por Cohorte"),

    html.Label("Selecciona facultad:"),
    dcc.Dropdown(
        id='facultad-des',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        placeholder="Selecciona una facultad"
    ),

    html.Label("Selecciona carrera:"),
    dcc.Dropdown(id='carrera-des'),

    html.Label("¿Solo empleo formal?"),
    dcc.Checklist(
        id='solo-formal-des',
        options=[{'label': 'Sí', 'value': 'formal'}],
        value=[]
    ),

    html.Label("Tipo de gráfico:"),
    dcc.RadioItems(
        id='tipo-grafico-des',
        options=[
            {'label': 'Líneas', 'value': 'line'},
            {'label': 'Barras', 'value': 'bar'}
        ],
        value='line',
        labelStyle={'display': 'inline-block', 'margin-right': '15px'}
    ),

    dcc.Graph(id='grafico-desempleo')
])

# === Actualizar carreras al cambiar facultad ===
@app.callback(
    Output('carrera-des', 'options'),
    Input('facultad-des', 'value')
)
def actualizar_carreras(facultad):
    if facultad:
        carreras = df[df['FACULTAD'] == facultad]['CarreraHomologada.1'].dropna().unique()
        return [{'label': c, 'value': c} for c in sorted(carreras)]
    return []

# === Callback para gráfico ===
@app.callback(
    Output('grafico-desempleo', 'figure'),
    Input('facultad-des', 'value'),
    Input('carrera-des', 'value'),
    Input('solo-formal-des', 'value'),
    Input('tipo-grafico-des', 'value')
)
def graficar_desempleo(facultad, carrera, solo_formal, tipo_grafico):
    data = df.copy()

    if facultad:
        data = data[data['FACULTAD'] == facultad]
    if carrera:
        data = data[data['CarreraHomologada.1'] == carrera]

    # Total: todos los graduados
    total = data.groupby('AnioGraduacion.1')['IdentificacionBanner.1'].nunique()

    # Empleados: según filtro
    if 'formal' in solo_formal:
        empleados_df = data[data['Empleo formal'].astype(str).str.upper() == 'EMPLEO FORMAL']
    else:
        empleados_df = data[data['Esta_empleado']]

    empleados = empleados_df.groupby('AnioGraduacion.1')['IdentificacionBanner.1'].nunique()

    # Unir y calcular desempleo
    resumen = pd.DataFrame({'empleados': empleados, 'total': total}).reset_index()
    resumen = resumen[resumen['total'] > 0]
    resumen['desempleo'] = 1 - (resumen['empleados'] / resumen['total'])
    resumen = resumen.sort_values('AnioGraduacion.1')

    titulo = f'Tasa de desempleo por cohorte {"(solo empleo formal)" if "formal" in solo_formal else ""}'

    # Gráfico
    if tipo_grafico == 'bar':
        fig = px.bar(resumen, x='AnioGraduacion.1', y='desempleo',
                     labels={'AnioGraduacion.1': 'Año de graduación', 'desempleo': 'Tasa de desempleo'},
                     title=titulo, text_auto='.1%')
    else:
        fig = px.line(resumen, x='AnioGraduacion.1', y='desempleo',
                      labels={'AnioGraduacion.1': 'Año de graduación', 'desempleo': 'Tasa de desempleo'},
                      title=titulo, markers=True)
        fig.update_traces(mode='lines+markers')
        fig.update_yaxes(tickformat=".0%")

    return fig

# === Ejecutar app ===
if __name__ == '__main__':
    app.run(debug=True)


Transiciones de empleo informal y formal

In [5]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# === 1. Cargar datos ===
df = pd.read_excel(r"data/empl.xlsx", engine="openpyxl")

# === 2. Preparar salario y meses ===
df['SALARIO.1'] = pd.to_numeric(df['SALARIO.1'], errors='coerce')
df = df[df['Mes.1'].isin([2, 5, 9, 11])]

# === 3. Conservar solo el empleo con mayor salario por persona y mes ===
duplicados = df.duplicated(subset=['IdentificacionBanner.1', 'Mes.1'], keep=False)
df_multi = df[duplicados].copy()
df_uniq = df[~duplicados].copy()

df_multi = df_multi.sort_values('SALARIO.1', ascending=False)
df_multi = df_multi.drop_duplicates(subset=['IdentificacionBanner.1', 'Mes.1'], keep='first')
df = pd.concat([df_uniq, df_multi], ignore_index=True)

# === 4. Clasificar empleo formal ===
df['Empleo formal'] = df['Empleo formal'].str.strip().str.upper()
df['Formal'] = df['Empleo formal'].map({'EMPLEO FORMAL': 1, 'EMPLEO NO FORMAL': 0})

# === 5. Iniciar la app Dash ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Transiciones de empleo por trimestre"),

    html.Label("Filtrar por Facultad:"),
    dcc.Dropdown(
        id='filtro_facultad',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        value=None,
        placeholder="Selecciona una facultad",
        multi=False
    ),

    html.Label("Filtrar por Carrera:"),
    dcc.Dropdown(
        id='filtro_carrera',
        options=[{'label': c, 'value': c} for c in sorted(df['CarreraHomologada.1'].dropna().unique())],
        value=None,
        placeholder="Selecciona una carrera",
        multi=False
    ),

    dcc.Graph(id='grafico_transiciones')
])

# === 6. Callback para actualizar gráfico ===
@app.callback(
    Output('grafico_transiciones', 'figure'),
    Input('filtro_facultad', 'value'),
    Input('filtro_carrera', 'value')
)
def actualizar_grafico(facultad, carrera):
    df_filtrado = df.copy()
    if facultad:
        df_filtrado = df_filtrado[df_filtrado['FACULTAD'] == facultad]
    if carrera:
        df_filtrado = df_filtrado[df_filtrado['CarreraHomologada.1'] == carrera]

    # Rehacer pivote por persona y mes
    pivot = df_filtrado.pivot(index='IdentificacionBanner.1', columns='Mes.1', values='Formal')
    if not set([2, 5, 9, 11]).issubset(pivot.columns):
        return px.bar(title="No hay suficientes datos para mostrar transiciones.")

    pivot = pivot[[2, 5, 9, 11]]
    pivot.columns = ['Feb', 'May', 'Sep', 'Nov']

    def clasificar(ant, act):
        if pd.isna(ant) or pd.isna(act):
            return 'Desconocido'
        elif ant == 1 and act == 1:
            return 'Permanece Formal'
        elif ant == 0 and act == 0:
            return 'Permanece Informal'
        elif ant == 1 and act == 0:
            return 'Pasa a Informal'
        elif ant == 0 and act == 1:
            return 'Pasa a Formal'
        return 'Desconocido'

    transiciones = []
    for (a, b), q in zip([('Feb', 'May'), ('May', 'Sep'), ('Sep', 'Nov')], ['Q1→Q2', 'Q2→Q3', 'Q3→Q4']):
        temp = pivot[[a, b]].copy()
        temp['Trimestre'] = q
        temp['Transición'] = temp.apply(lambda row: clasificar(row[a], row[b]), axis=1)
        transiciones.append(temp[['Trimestre', 'Transición']])

    df_trans = pd.concat(transiciones)
    conteo = df_trans.groupby(['Trimestre', 'Transición']).size().reset_index(name='Cantidad')

    fig = px.bar(
        conteo,
        x='Trimestre',
        y='Cantidad',
        color='Transición',
        text='Cantidad',
        title='Transiciones de empleo por trimestre'
    )
    fig.update_layout(barmode='stack', yaxis_title='Número de graduados', xaxis_title='Trimestre')
    return fig

# === 7. Ejecutar la app ===
if __name__ == '__main__':
    app.run(debug=True)


Tiempo al primer empleo

In [2]:
pip install dash pandas plotly openpyxl python-dateutil


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from dateutil.relativedelta import relativedelta

# === 1. Cargar datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx", engine="openpyxl")

# === 2. Limpiar fechas relevantes ===
df['FechaGraduacion.1'] = pd.to_datetime(df['FechaGraduacion.1'], errors='coerce')
df['FECINGAFI.1'] = pd.to_datetime(df['FECINGAFI.1'], errors='coerce')

# Eliminar registros sin fechas válidas
df = df.dropna(subset=['FechaGraduacion.1', 'FECINGAFI.1'])

# Eliminar registros donde el empleo es antes de graduarse
df = df[df['FECINGAFI.1'] >= df['FechaGraduacion.1']]

# === 3. Calcular meses exactos al primer empleo ===
def calcular_meses(graduacion, ingreso):
    delta = relativedelta(ingreso, graduacion)
    return delta.years * 12 + delta.months

df['Meses al primer empleo'] = df.apply(
    lambda row: calcular_meses(row['FechaGraduacion.1'], row['FECINGAFI.1']),
    axis=1
)

# Conservar solo el primer empleo por persona
df = df.sort_values(['IdentificacionBanner.1', 'FECINGAFI.1'])
df = df.drop_duplicates(subset='IdentificacionBanner.1', keep='first')

# === 4. Iniciar app Dash ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Tiempo exacto al primer empleo desde la graduación"),

    html.Label("Filtrar por Facultad:"),
    dcc.Dropdown(
        id='filtro_facultad',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        value=None,
        placeholder="Selecciona una facultad"
    ),

    html.Label("Filtrar por Carrera:"),
    dcc.Dropdown(
        id='filtro_carrera',
        options=[{'label': c, 'value': c} for c in sorted(df['CarreraHomologada.1'].dropna().unique())],
        value=None,
        placeholder="Selecciona una carrera"
    ),

    dcc.Graph(id='grafico_tiempo_empleo')
])

# === 5. Callback para graficar ===
@app.callback(
    Output('grafico_tiempo_empleo', 'figure'),
    Input('filtro_facultad', 'value'),
    Input('filtro_carrera', 'value')
)
def actualizar_grafico(facultad, carrera):
    datos = df.copy()
    if facultad:
        datos = datos[datos['FACULTAD'] == facultad]
    if carrera:
        datos = datos[datos['CarreraHomologada.1'] == carrera]

    if datos.empty:
        return px.bar(title="No hay datos disponibles para esta selección.")

    fig = px.histogram(
        datos,
        x='Meses al primer empleo',
        nbins=20,
        title='Distribución del tiempo al primer empleo',
        labels={'Meses al primer empleo': 'Meses desde graduación'}
    )
    fig.update_layout(yaxis_title='Número de graduados', xaxis_title='Meses desde graduación')
    return fig

# === 6. Ejecutar ===
if __name__ == '__main__':
    app.run(debug=True)


Conexiones con empresas clave

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# === 1. Cargar datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx", engine="openpyxl")

# Limpieza básica
df['Empleo formal'] = df['Empleo formal'].str.strip().str.upper()
df['NOMEMP.1'] = df['NOMEMP.1'].fillna('SIN EMPRESA')
df['Cantidad de empleados'] = pd.to_numeric(df['Cantidad de empleados'], errors='coerce')

# === 2. Iniciar la app Dash ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Conexiones con empresas clave"),

    html.Label("Filtrar por Facultad:"),
    dcc.Dropdown(
        id='filtro_facultad',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        value=None,
        placeholder="Selecciona una facultad"
    ),

    html.Label("Filtrar por Carrera:"),
    dcc.Dropdown(
        id='filtro_carrera',
        options=[{'label': c, 'value': c} for c in sorted(df['CarreraHomologada.1'].dropna().unique())],
        value=None,
        placeholder="Selecciona una carrera"
    ),

    html.Label("Filtrar por tipo de empleo:"),
    dcc.Dropdown(
        id='filtro_formalidad',
        options=[
            {'label': 'Todos', 'value': 'TODOS'},
            {'label': 'Empleo Formal', 'value': 'EMPLEO FORMAL'},
            {'label': 'Empleo No Formal', 'value': 'EMPLEO NO FORMAL'},
        ],
        value='TODOS'
    ),

    html.Label("Filtrar por Sector Económico:"),
    dcc.Dropdown(
        id='filtro_sector',
        options=[{'label': s, 'value': s} for s in sorted(df['SECTOR'].dropna().unique())],
        value=None,
        placeholder="Selecciona un sector económico"
    ),

    html.Label("Filtrar por Tamaño de Empresa (Cantidad de empleados):"),
    dcc.RangeSlider(
        id='filtro_tamano',
        min=0,
        max=int(df['Cantidad de empleados'].max(skipna=True)),
        step=1,
        value=[0, int(df['Cantidad de empleados'].max(skipna=True))],
        tooltip={"placement": "bottom", "always_visible": False}
    ),

    dcc.Graph(id='grafico_empresas')
])

# === 3. Callback para actualizar gráfico ===
@app.callback(
    Output('grafico_empresas', 'figure'),
    Input('filtro_facultad', 'value'),
    Input('filtro_carrera', 'value'),
    Input('filtro_formalidad', 'value'),
    Input('filtro_sector', 'value'),
    Input('filtro_tamano', 'value')
)
def actualizar_grafico(facultad, carrera, formalidad, sector, tamano_rango):
    datos = df.copy()

    if facultad:
        datos = datos[datos['FACULTAD'] == facultad]
    if carrera:
        datos = datos[datos['CarreraHomologada.1'] == carrera]
    if formalidad != 'TODOS':
        datos = datos[datos['Empleo formal'] == formalidad]
    if sector:
        datos = datos[datos['SECTOR'] == sector]
    datos = datos[(datos['Cantidad de empleados'] >= tamano_rango[0]) &
                  (datos['Cantidad de empleados'] <= tamano_rango[1])]

    top_empresas = datos['NOMEMP.1'].value_counts().nlargest(10).reset_index()
    top_empresas.columns = ['Empresa', 'Contrataciones']

    if top_empresas.empty:
        return px.bar(title="No hay datos disponibles para esta combinación de filtros.")

    fig = px.bar(
        top_empresas,
        x='Empresa',
        y='Contrataciones',
        text='Contrataciones',
        title='Top 10 empresas que contratan graduados',
    )
    fig.update_layout(yaxis_title='Número de graduados contratados')
    return fig

# === 4. Ejecutar la app ===
if __name__ == '__main__':
    app.run(debug=True)


Cargos

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# === 1. Cargar datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx", engine="openpyxl")

# Limpieza básica
df['Empleo formal'] = df['Empleo formal'].str.strip().str.upper()
df['SALARIO.1'] = pd.to_numeric(df['SALARIO.1'], errors='coerce')
df['OCUAFI.1'] = df['OCUAFI.1'].fillna('SIN INFORMACIÓN')

# === 2. Iniciar Dash App ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Ranking de cargos ocupados por graduados"),

    html.Label("Filtrar por Facultad:"),
    dcc.Dropdown(
        id='filtro_facultad',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        value=None,
        placeholder="Selecciona una facultad"
    ),

    html.Label("Filtrar por Carrera:"),
    dcc.Dropdown(
        id='filtro_carrera',
        options=[{'label': c, 'value': c} for c in sorted(df['CarreraHomologada.1'].dropna().unique())],
        value=None,
        placeholder="Selecciona una carrera"
    ),

    html.Label("Filtrar por tipo de empleo:"),
    dcc.Dropdown(
        id='filtro_formalidad',
        options=[
            {'label': 'Todos', 'value': 'TODOS'},
            {'label': 'Empleo Formal', 'value': 'EMPLEO FORMAL'},
            {'label': 'Empleo No Formal', 'value': 'EMPLEO NO FORMAL'},
        ],
        value='TODOS'
    ),

    dcc.Graph(id='grafico_cargos')
])

# === 3. Callback ===
@app.callback(
    Output('grafico_cargos', 'figure'),
    Input('filtro_facultad', 'value'),
    Input('filtro_carrera', 'value'),
    Input('filtro_formalidad', 'value')
)
def actualizar_grafico(facultad, carrera, formalidad):
    datos = df.copy()
    if facultad:
        datos = datos[datos['FACULTAD'] == facultad]
    if carrera:
        datos = datos[datos['CarreraHomologada.1'] == carrera]
    if formalidad != 'TODOS':
        datos = datos[datos['Empleo formal'] == formalidad]

    # Agrupar por cargo
    resumen = datos.groupby('OCUAFI.1').agg(
        Total=('OCUAFI.1', 'count'),
        SalarioPromedio=('SALARIO.1', 'mean')
    ).sort_values('Total', ascending=False).reset_index().head(15)

    if resumen.empty:
        return px.bar(title="No hay datos disponibles para esta combinación de filtros.")

    fig = px.bar(
        resumen,
        x='OCUAFI.1',
        y='Total',
        text='Total',
        hover_data={'SalarioPromedio': ':.2f'},
        title='Top 15 cargos ocupados por graduados'
    )
    fig.update_layout(
        xaxis_title='Cargo',
        yaxis_title='Número de graduados',
        xaxis_tickangle=-45
    )
    return fig

# === 4. Ejecutar la app ===
if __name__ == '__main__':
    app.run(debug=True)


Mapa Geográfico de Empleabilidad

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# === 1. Cargar datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx", engine="openpyxl")

# === 2. Procesar coordenadas ===
df = df[df['COORDENADA'].notna()]
df[['lat', 'lon']] = df['COORDENADA'].str.split(',', expand=True)
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

# Filtrar solo coordenadas válidas
df = df.dropna(subset=['lat', 'lon'])

# Procesar campos clave
df['Empleo formal'] = df['Empleo formal'].str.strip().str.upper()
df['SALARIO.1'] = pd.to_numeric(df['SALARIO.1'], errors='coerce')

# === 3. Iniciar app ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Mapa de empleabilidad de graduados"),

    html.Label("Filtrar por Facultad:"),
    dcc.Dropdown(
        id='filtro_facultad',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        value=None,
        placeholder="Selecciona una facultad"
    ),

    html.Label("Filtrar por Carrera:"),
    dcc.Dropdown(
        id='filtro_carrera',
        options=[{'label': c, 'value': c} for c in sorted(df['CarreraHomologada.1'].dropna().unique())],
        value=None,
        placeholder="Selecciona una carrera"
    ),

    html.Label("Filtrar por tipo de empleo:"),
    dcc.Dropdown(
        id='filtro_formalidad',
        options=[
            {'label': 'Todos', 'value': 'TODOS'},
            {'label': 'Empleo Formal', 'value': 'EMPLEO FORMAL'},
            {'label': 'Empleo No Formal', 'value': 'EMPLEO NO FORMAL'},
        ],
        value='TODOS'
    ),

    dcc.Graph(id='mapa_empleabilidad')
])

# === 4. Callback para mapa ===
@app.callback(
    Output('mapa_empleabilidad', 'figure'),
    Input('filtro_facultad', 'value'),
    Input('filtro_carrera', 'value'),
    Input('filtro_formalidad', 'value')
)
def actualizar_mapa(facultad, carrera, formalidad):
    datos = df.copy()

    if facultad:
        datos = datos[datos['FACULTAD'] == facultad]
    if carrera:
        datos = datos[datos['CarreraHomologada.1'] == carrera]
    if formalidad != 'TODOS':
        datos = datos[datos['Empleo formal'] == formalidad]

    if datos.empty:
        return px.scatter_mapbox(lat=[], lon=[], title="No hay datos geográficos para mostrar")

    fig = px.scatter_mapbox(
        datos,
        lat='lat',
        lon='lon',
        hover_name='NOMEMP.1',
        hover_data={
            'Ciudad': datos['CIUDAD'],
            'Provincia': datos['PROVINCIA'],
            'SALARIO.1': ':.2f',
            'lat': False,
            'lon': False
        },
        zoom=5,
        height=600
    )
    fig.update_layout(
        mapbox_style='open-street-map',
        title="Ubicación geográfica de los empleos de graduados",
        margin={'r':0, 't':40, 'l':0, 'b':0}
    )
    return fig

# === 5. Ejecutar ===
if __name__ == '__main__':
    app.run(debug=True)


: 

MAPA POR SALARIOS

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# === 1. Cargar datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx", engine="openpyxl")

# Procesar coordenadas
df = df[df['COORDENADA'].notna()]
df[['lat', 'lon']] = df['COORDENADA'].str.split(',', expand=True)
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')
df = df.dropna(subset=['lat', 'lon'])

# Limpieza adicional
df['Empleo formal'] = df['Empleo formal'].str.strip().str.upper()
df['SALARIO.1'] = pd.to_numeric(df['SALARIO.1'], errors='coerce')
df['CIUDAD'] = df['CIUDAD'].fillna('DESCONOCIDA')

# === 2. Iniciar Dash App ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Mapa de empleabilidad por ciudad - Promedio salarial"),

    html.Label("Filtrar por Facultad:"),
    dcc.Dropdown(
        id='filtro_facultad',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        value=None,
        placeholder="Selecciona una facultad"
    ),

    html.Label("Filtrar por Carrera:"),
    dcc.Dropdown(
        id='filtro_carrera',
        options=[{'label': c, 'value': c} for c in sorted(df['CarreraHomologada.1'].dropna().unique())],
        value=None,
        placeholder="Selecciona una carrera"
    ),

    html.Label("Filtrar por tipo de empleo:"),
    dcc.Dropdown(
        id='filtro_formalidad',
        options=[
            {'label': 'Todos', 'value': 'TODOS'},
            {'label': 'Empleo Formal', 'value': 'EMPLEO FORMAL'},
            {'label': 'Empleo No Formal', 'value': 'EMPLEO NO FORMAL'},
        ],
        value='TODOS'
    ),

    dcc.Graph(id='mapa_promedio_ciudad')
])

# === 3. Callback ===
@app.callback(
    Output('mapa_promedio_ciudad', 'figure'),
    Input('filtro_facultad', 'value'),
    Input('filtro_carrera', 'value'),
    Input('filtro_formalidad', 'value')
)
def actualizar_mapa(facultad, carrera, formalidad):
    datos = df.copy()

    if facultad:
        datos = datos[datos['FACULTAD'] == facultad]
    if carrera:
        datos = datos[datos['CarreraHomologada.1'] == carrera]
    if formalidad != 'TODOS':
        datos = datos[datos['Empleo formal'] == formalidad]

    if datos.empty:
        return px.scatter_mapbox(lat=[], lon=[], title="No hay datos geográficos para mostrar")

    # Agrupar por ciudad
    resumen = datos.groupby('CIUDAD').agg(
        Total=('CIUDAD', 'count'),
        SalarioPromedio=('SALARIO.1', 'mean'),
        lat=('lat', 'mean'),
        lon=('lon', 'mean')
    ).reset_index()

    fig = px.scatter_mapbox(
        resumen,
        lat='lat',
        lon='lon',
        size='Total',
        color='SalarioPromedio',
        hover_name='CIUDAD',
        hover_data={
            'Total': True,
            'SalarioPromedio': ':.2f',
            'lat': False,
            'lon': False
        },
        zoom=5,
        height=600,
        color_continuous_scale='Viridis',
        size_max=30
    )
    fig.update_layout(
        mapbox_style='open-street-map',
        title="Salario promedio y empleabilidad por ciudad",
        margin={'r':0, 't':40, 'l':0, 'b':0}
    )
    return fig

# === 4. Ejecutar ===
if __name__ == '__main__':
    app.run(debug=True)


Duracion de empleo

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from dateutil.relativedelta import relativedelta

# === 1. Cargar datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx", engine="openpyxl")

# === 2. Procesamiento inicial ===
df['FECINGAFI.1'] = pd.to_datetime(df['FECINGAFI.1'], errors='coerce')
df['Empleo formal'] = df['Empleo formal'].str.strip().str.upper()
df = df.dropna(subset=['FECINGAFI.1', 'IdentificacionBanner.1', 'NOMEMP.1'])

# Ordenar y calcular duración
df = df.sort_values(['IdentificacionBanner.1', 'NOMEMP.1', 'FECINGAFI.1'])

empleos = []

for _, grupo in df.groupby(['IdentificacionBanner.1', 'NOMEMP.1']):
    fechas = grupo['FECINGAFI.1'].tolist()
    for i in range(len(fechas) - 1):
        inicio = fechas[i]
        fin = fechas[i + 1]
        delta = relativedelta(fin, inicio)
        duracion_meses = delta.years * 12 + delta.months
        if duracion_meses > 0:
            registro = grupo.iloc[i].to_dict()
            registro['DuracionMeses'] = duracion_meses
            empleos.append(registro)

df_duracion = pd.DataFrame(empleos)

# === 3. Dash app ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Duración de empleos de graduados"),

    html.Label("Filtrar por Facultad:"),
    dcc.Dropdown(
        id='filtro_facultad',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        value=None,
        placeholder="Selecciona una facultad"
    ),

    html.Label("Filtrar por Carrera:"),
    dcc.Dropdown(
        id='filtro_carrera',
        options=[{'label': c, 'value': c} for c in sorted(df['CarreraHomologada.1'].dropna().unique())],
        value=None,
        placeholder="Selecciona una carrera"
    ),

    html.Label("Filtrar por tipo de empleo:"),
    dcc.Dropdown(
        id='filtro_formalidad',
        options=[
            {'label': 'Todos', 'value': 'TODOS'},
            {'label': 'Empleo Formal', 'value': 'EMPLEO FORMAL'},
            {'label': 'Empleo No Formal', 'value': 'EMPLEO NO FORMAL'},
        ],
        value='TODOS'
    ),

    dcc.RadioItems(
        id='tipo_grafico',
        options=[
            {'label': 'Histograma', 'value': 'hist'},
            {'label': 'Boxplot por carrera', 'value': 'box'}
        ],
        value='hist',
        labelStyle={'display': 'inline-block', 'margin-right': '15px'}
    ),

    dcc.Graph(id='grafico_duracion')
])

# === 4. Callback ===
@app.callback(
    Output('grafico_duracion', 'figure'),
    Input('filtro_facultad', 'value'),
    Input('filtro_carrera', 'value'),
    Input('filtro_formalidad', 'value'),
    Input('tipo_grafico', 'value')
)
def actualizar_grafico(facultad, carrera, formalidad, tipo):
    datos = df_duracion.copy()
    if facultad:
        datos = datos[datos['FACULTAD'] == facultad]
    if carrera:
        datos = datos[datos['CarreraHomologada.1'] == carrera]
    if formalidad != 'TODOS':
        datos = datos[datos['Empleo formal'] == formalidad]

    if datos.empty:
        return px.bar(title="No hay datos para esta combinación de filtros.")

    if tipo == 'hist':
        fig = px.histogram(
            datos,
            x='DuracionMeses',
            nbins=20,
            title='Distribución de duración de empleos',
            labels={'DuracionMeses': 'Meses'}
        )
        fig.update_layout(yaxis_title='Cantidad de empleos')
    else:
        fig = px.box(
            datos,
            x='CarreraHomologada.1',
            y='DuracionMeses',
            title='Duración de empleo por carrera',
            labels={'DuracionMeses': 'Meses'},
            points='all'
        )
        fig.update_layout(xaxis_tickangle=-45)

    return fig

# === 5. Ejecutar ===
if __name__ == '__main__':
    app.run(debug=True)


indice de rotación

In [4]:
pip install pandas openpyxl python-dateutil


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from dateutil.relativedelta import relativedelta

# === CARGA DE DATOS ===
archivo = r"data/empleabilidad.xlsx"
df = pd.read_excel(archivo, engine="openpyxl")

# === PREPARACION ===
df['FECINGAFI.1'] = pd.to_datetime(df['FECINGAFI.1'], errors='coerce')
df = df.dropna(subset=['FECINGAFI.1', 'IdentificacionBanner.1', 'NOMEMP.1'])
df = df.sort_values(by=['IdentificacionBanner.1', 'FECINGAFI.1'])

rotaciones = []

# === DETECCION DE ROTACION ===
for pid, grupo in df.groupby('IdentificacionBanner.1'):
    empresas = grupo['NOMEMP.1'].tolist()
    fechas = grupo['FECINGAFI.1'].tolist()
    carrera = grupo['CarreraHomologada.1'].iloc[0]
    facultad = grupo['FACULTAD'].iloc[0]
    empleo_formal = grupo['Empleo formal'].iloc[0]

    rotado = 0
    if len(empresas) > 1:
        primera_fecha = fechas[0]
        for i in range(1, len(empresas)):
            if empresas[i] != empresas[i-1]:
                delta = relativedelta(fechas[i], primera_fecha)
                meses = delta.years * 12 + delta.months
                if meses <= 12:
                    rotado = 1
                    break

    rotaciones.append({
        'IdentificacionBanner.1': pid,
        'CarreraHomologada.1': carrera,
        'FACULTAD': facultad,
        'Empleo formal': empleo_formal,
        'Rotacion': rotado
    })

# === AGRUPAR RESULTADOS ===
df_rotacion = pd.DataFrame(rotaciones)

resumen = df_rotacion.groupby('CarreraHomologada.1').agg(
    Total=('IdentificacionBanner.1', 'count'),
    ConRotacion=('Rotacion', 'sum')
).reset_index()

resumen['TasaRotacion'] = resumen['ConRotacion'] / resumen['Total'] * 100

# === MOSTRAR RESULTADOS ===
print(resumen.sort_values(by='TasaRotacion', ascending=False))


                                  CarreraHomologada.1  Total  ConRotacion  \
34                    MAESTRIA EN CIENCIAS BIOMEDICAS      3            1   
7                                      CIBERSEGURIDAD     45           10   
79                                           MEDICINA   1246          242   
16                       ELECTRONICA Y AUTOMATIZACION     23            4   
17                                         ENFERMERIA    816          134   
..                                                ...    ...          ...   
44         MAESTRIA EN DISEÑO ARQUITECTONICO AVANZADO     18            0   
43  MAESTRIA EN DIRECCION Y POSTPRODUCCION AUDIOVI...     17            0   
48                 MAESTRIA EN EXPERIENCIA DE USUARIO      1            0   
93                     TECNICO SUPERIOR EN OBRA CIVIL      3            0   
98                             TECNOLOGIA EN FINANZAS     15            0   

    TasaRotacion  
34     33.333333  
7      22.222222  
79     19.422151  

In [ ]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# === CARGA DE DATOS ===
df = pd.read_excel(r"data/empleabilidad.xlsx", engine="openpyxl")
df['FECINGAFI.1'] = pd.to_datetime(df['FECINGAFI.1'], errors='coerce')
df = df.dropna(subset=['FECINGAFI.1', 'IdentificacionBanner.1', 'NOMEMP.1'])
df = df.sort_values(by=['IdentificacionBanner.1', 'FECINGAFI.1'])

# === FUNCION PARA CALCULAR ROTACION ===
def calcular_rotacion(df_filtrado):
    rotaciones = []
    for pid, grupo in df_filtrado.groupby('IdentificacionBanner.1'):
        empresas = grupo['NOMEMP.1'].tolist()
        fechas = grupo['FECINGAFI.1'].tolist()
        carrera = grupo['CarreraHomologada.1'].iloc[0]
        facultad = grupo['FACULTAD'].iloc[0]
        empleo_formal = grupo['Empleo formal'].iloc[0]

        rotado = 0
        if len(empresas) > 1:
            primera_fecha = fechas[0]
            for i in range(1, len(empresas)):
                if empresas[i] != empresas[i - 1]:
                    delta = relativedelta(fechas[i], primera_fecha)
                    meses = delta.years * 12 + delta.months
                    if meses <= 12:
                        rotado = 1
                        break

        rotaciones.append({
            'IdentificacionBanner.1': pid,
            'CarreraHomologada.1': carrera,
            'FACULTAD': facultad,
            'Empleo formal': empleo_formal,
            'Rotacion': rotado
        })

    df_rotacion = pd.DataFrame(rotaciones)
    resumen = df_rotacion.groupby('CarreraHomologada.1').agg(
        Total=('IdentificacionBanner.1', 'count'),
        ConRotacion=('Rotacion', 'sum')
    ).reset_index()
    resumen['TasaRotacion'] = resumen['ConRotacion'] / resumen['Total'] * 100
    return resumen

# === APP DASH ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Índice de rotación en el primer año por carrera"),

    html.Label("Filtrar por Facultad:"),
    dcc.Dropdown(
        id='filtro_facultad',
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        value=None,
        placeholder="Selecciona una facultad"
    ),

    html.Label("Filtrar por tipo de empleo:"),
    dcc.Dropdown(
        id='filtro_empleo',
        options=[
            {'label': 'Todos', 'value': 'TODOS'},
            {'label': 'Empleo Formal', 'value': 'EMPLEO FORMAL'},
            {'label': 'Empleo No Formal', 'value': 'EMPLEO NO FORMAL'}
        ],
        value='TODOS'
    ),

    dcc.Graph(id='grafico_rotacion')
])

@app.callback(
    Output('grafico_rotacion', 'figure'),
    Input('filtro_facultad', 'value'),
    Input('filtro_empleo', 'value')
)
def actualizar_grafico(facultad, empleo):
    data = df.copy()
    if facultad:
        data = data[data['FACULTAD'] == facultad]
    if empleo != 'TODOS':
        data = data[data['Empleo formal'].str.upper().str.strip() == empleo]

    if data.empty:
        return px.bar(title="No hay datos para esta combinación de filtros")

    resumen = calcular_rotacion(data)
    if resumen.empty:
        return px.bar(title="No hay datos para esta combinación de filtros")

    fig = px.bar(
        resumen.sort_values(by='TasaRotacion', ascending=False),
        x='CarreraHomologada.1',
        y='TasaRotacion',
        title='Tasa de rotación en el primer año por carrera',
        labels={'CarreraHomologada.1': 'Carrera', 'TasaRotacion': 'Rotación (%)'},
        text='TasaRotacion'
    )
    fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    fig.update_layout(xaxis_tickangle=-45, yaxis_title="Rotación (%)", height=600)
    return fig

if __name__ == '__main__':
    app.run(debug=True)


Movilidad sectoorial

In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

# === 1. Cargar datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx")
df = df.dropna(subset=['SECTOR', 'FECINGAFI.1'])
df['FECINGAFI.1'] = pd.to_datetime(df['FECINGAFI.1'], errors='coerce')

# === 2. Ordenar y detectar transiciones
df = df.sort_values(by=['IdentificacionBanner', 'FECINGAFI.1'])
df['sector_anterior'] = df.groupby('IdentificacionBanner')['SECTOR'].shift()
df['sector_actual'] = df['SECTOR']
df = df.dropna(subset=['sector_anterior', 'sector_actual'])
df = df[df['sector_anterior'] != df['sector_actual']]

# === 3. Función de flujo
def generar_flujo(df_filtrado):
    transiciones = df_filtrado.groupby(['sector_anterior', 'sector_actual']).size().reset_index(name='count')
    if transiciones.empty:
        return [], [], [], []

    labels = pd.unique(transiciones[['sector_anterior', 'sector_actual']].values.ravel())
    label_dict = {label: idx for idx, label in enumerate(labels)}
    source = transiciones['sector_anterior'].map(label_dict)
    target = transiciones['sector_actual'].map(label_dict)
    value = transiciones['count']

    return labels, source.tolist(), target.tolist(), value.tolist()

# === 4. App Dash ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Movilidad intersectorial"),

    html.Label("Filtrar por facultad:"),
    dcc.Dropdown(
        options=[{'label': i, 'value': i} for i in df['FACULTAD'].dropna().unique()],
        id='facultad_dropdown',
        placeholder='Selecciona una facultad'
    ),

    html.Label("Filtrar por empleo:"),
    dcc.Dropdown(
        options=[
            {'label': 'TODOS', 'value': 'TODOS'},
            {'label': 'EMPLEO FORMAL', 'value': 'EMPLEO FORMAL'},
            {'label': 'EMPLEO NO FORMAL', 'value': 'EMPLEO NO FORMAL'},
        ],
        id='empleo_dropdown',
        value='TODOS'
    ),

    dcc.Graph(id='sankey_sector')
])

@app.callback(
    Output('sankey_sector', 'figure'),
    Input('facultad_dropdown', 'value'),
    Input('empleo_dropdown', 'value')
)
def actualizar_sankey(facultad, empleo):
    data = df.copy()
    if facultad:
        data = data[data['FACULTAD'] == facultad]
    if empleo != 'TODOS':
        data = data[data['Empleo formal'] == empleo]

    labels, source, target, value = generar_flujo(data)

    if len(labels) == 0:
        return go.Figure().update_layout(title="No hay datos suficientes para mostrar flujo")

    fig = go.Figure(data=[go.Sankey(
        node=dict(pad=15, thickness=20, line=dict(color="black", width=0.5), label=labels),
        link=dict(source=source, target=target, value=value)
    )])

    fig.update_layout(title_text="Transiciones entre sectores", font_size=10)
    return fig

# === 5. Ejecutar ===
if __name__ == '__main__':
    app.run(debug=True)


Alerta de saturacipon del mercado

In [19]:
pip install dash-bootstrap-components


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, dash_table

# === 1. Cargar datos ===
df = pd.read_excel(r"data/empleabilidad.xlsx")

# === 2. Preparar datos base ===
df['TieneSalario'] = df['SALARIO.1'].notna()
df['Periodo'] = df['AnioGraduacion'].astype(str)  # o ajusta si tienes otra lógica para periodo

# === 3. Crear app Dash ===
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Alertas de saturación del mercado laboral"),
    
    html.Label("Filtrar por facultad:"),
    dcc.Dropdown(
        options=[{'label': f, 'value': f} for f in sorted(df['FACULTAD'].dropna().unique())],
        id='facultad_dropdown_alerta',
        placeholder='Selecciona una facultad'
    ),
    
    html.Label("Filtrar por carrera:"),
    dcc.Dropdown(id='carrera_dropdown_alerta', placeholder='Selecciona una carrera'),

    dash_table.DataTable(
        id='tabla_alertas',
        columns=[
            {"name": i, "id": i} for i in [
                'CarreraHomologada.1', 'Periodo', 'Cantidad Graduados',
                'Con salario (%)', 'Salario Promedio', 'Formales', 'No Formales', 'Alerta Saturación'
            ]
        ],
        style_table={'overflowX': 'auto'},
        style_data_conditional=[
            {
                'if': {
                    'filter_query': '{Alerta Saturación} = "Alta Saturación"',
                    'column_id': 'Alerta Saturación'
                },
                'backgroundColor': 'tomato',
                'color': 'white'
            }
        ],
        page_size=10
    )
])

# === 4. Lógica de actualización de carrera y tabla ===

@app.callback(
    Output('carrera_dropdown_alerta', 'options'),
    Input('facultad_dropdown_alerta', 'value')
)
def actualizar_dropdown_carreras(facultad):
    if facultad is None:
        return []
    carreras = df[df['FACULTAD'] == facultad]['CarreraHomologada.1'].dropna().unique()
    return [{'label': c, 'value': c} for c in sorted(carreras)]

@app.callback(
    Output('tabla_alertas', 'data'),
    Input('facultad_dropdown_alerta', 'value'),
    Input('carrera_dropdown_alerta', 'value')
)
def generar_alertas(facultad=None, carrera=None):
    dff = df.copy()
    if facultad:
        dff = dff[dff['FACULTAD'] == facultad]
    if carrera:
        dff = dff[dff['CarreraHomologada.1'] == carrera]

    if dff.empty:
        return []

    resumen = dff.groupby(['CarreraHomologada.1', 'Periodo']).agg({
        'IdentificacionBanner': 'nunique',
        'TieneSalario': lambda x: np.mean(x) * 100,
        'SALARIO.1': 'mean',
        'Empleo formal': lambda x: dff.loc[x.index].loc[x == 'EMPLEO FORMAL', 'IdentificacionBanner'].nunique()
    }).reset_index()

    resumen.rename(columns={
        'IdentificacionBanner': 'Cantidad Graduados',
        'SALARIO.1': 'Salario Promedio'
    }, inplace=True)

    resumen['Formales'] = resumen['Empleo formal']
    resumen['No Formales'] = resumen['Cantidad Graduados'] - resumen['Formales']
    resumen['Con salario (%)'] = resumen['TieneSalario'].round(1)

    # === 5. Alerta de saturación ===
    def alerta(row):
        if row['Cantidad Graduados'] > 50 and row['No Formales'] / row['Cantidad Graduados'] > 0.5:
            return 'Alta Saturación'
        elif row['Cantidad Graduados'] > 30 and row['No Formales'] / row['Cantidad Graduados'] > 0.3:
            return 'Media Saturación'
        else:
            return 'Sin Alerta'

    resumen['Alerta Saturación'] = resumen.apply(alerta, axis=1)
    resumen.drop(columns=['TieneSalario', 'Empleo formal'], inplace=True)

    return resumen.to_dict('records')

# === 6. Ejecutar app ===
if __name__ == '__main__':
    app.run(debug=True)


In [1]:
import pandas as pd
import dash
from dash import html, dcc, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

# === 1. Cargar y preparar el DataFrame (ajusta la ruta)
df = pd.read_excel(r"data/empl.xlsx", sheet_name="Titulos")
df.columns = df.columns.str.upper().str.strip()
df["FECHA DE REGISTRO"] = pd.to_datetime(
    df["FECHA DE REGISTRO"], dayfirst=True, errors="coerce"
)
df["NIVEL ACADÉMICA"] = df["NIVEL ACADÉMICA"].str.upper().str.strip()
df["TIPO_TITULO"] = df["NIVEL ACADÉMICA"].apply(
    lambda x: "Pregrado" if "TERCER" in x else "Posgrado"
)

# === 2. Generar continuidad
df_sorted = df.sort_values(["IDENTIFICACION", "FECHA DE REGISTRO"])
df_sorted["ORDEN"] = df_sorted.groupby("IDENTIFICACION").cumcount() + 1

# Extraer Pregrado y Posgrado
pregrados = (
    df_sorted[df_sorted["TIPO_TITULO"] == "Pregrado"]
    .groupby("IDENTIFICACION")
    .first()
    .reset_index()
)
pregrados = pregrados.rename(
    columns={"TÍTULO HOMOLOGADO": "PREGRADO", "FECHA DE REGISTRO": "FECHA_PREGRADO"}
)

posgrados = df_sorted[df_sorted["TIPO_TITULO"] == "Posgrado"].copy()
posgrados["ORDEN"] = posgrados.groupby("IDENTIFICACION").cumcount() + 1
df_continuidad = posgrados.merge(
    pregrados[["IDENTIFICACION", "PREGRADO", "FECHA_PREGRADO"]],
    on="IDENTIFICACION",
    how="left",
)
df_continuidad["TIEMPO_DESDE_PREGRADO"] = (
    df_continuidad["FECHA DE REGISTRO"] - df_continuidad["FECHA_PREGRADO"]
).dt.days / 365.25
df_continuidad = df_continuidad.rename(
    columns={
        "INSTITUCIÓN DE EDUCACIÓN SUPERIOR": "POS_UNIVERSIDAD",
        "TÍTULO HOMOLOGADO": "POSGRADO",
    }
)

# === 3. Inicializar app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# === 4. Layout
app.layout = dbc.Container(
    [
        html.H2("Análisis de Continuidad y Recompra Educativa", className="my-4"),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label("Filtrar por Carrera de Pregrado:"),
                        dcc.Dropdown(
                            id="filtro_pregrado",
                            options=[
                                {"label": p, "value": p}
                                for p in sorted(
                                    df_continuidad["PREGRADO"].dropna().unique()
                                )
                            ],
                            placeholder="Selecciona una carrera",
                        ),
                    ],
                    width=6,
                )
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tasa de Continuidad"),
                            dbc.CardBody(
                                html.H4(id="tasa_continuidad", className="card-title")
                            ),
                        ]
                    )
                ),
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tasa de Recompra UDLA"),
                            dbc.CardBody(
                                html.H4(id="tasa_recompra", className="card-title")
                            ),
                        ]
                    )
                ),
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tiempo al 1er Posgrado"),
                            dbc.CardBody(
                                html.H4(id="tiempo_1er", className="card-title")
                            ),
                        ]
                    )
                ),
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tiempo al 2do Posgrado"),
                            dbc.CardBody(
                                html.H4(id="tiempo_2do", className="card-title")
                            ),
                        ]
                    )
                ),
            ],
            className="mb-4",
        ),
        dcc.Graph(id="grafico_top_especialidades"),
    ],
    fluid=True,
)


# === 5. Callbacks dinámicos
@app.callback(
    [
        Output("tasa_continuidad", "children"),
        Output("tasa_recompra", "children"),
        Output("tiempo_1er", "children"),
        Output("tiempo_2do", "children"),
        Output("grafico_top_especialidades", "figure"),
    ],
    [Input("filtro_pregrado", "value")],
)
def actualizar_datos(pregrado):
    if pregrado:
        data = df_continuidad[df_continuidad["PREGRADO"] == pregrado]
        base = df[df["TÍTULO HOMOLOGADO"] == pregrado]
    else:
        data = df_continuidad
        base = df[df["TIPO_TITULO"] == "Pregrado"]

    t_cont = round(
        100 * data["IDENTIFICACION"].nunique() / base["IDENTIFICACION"].nunique(), 1
    )
    t_recompra = round(
        100
        * data[data["POS_UNIVERSIDAD"] == "UNIVERSIDAD DE LAS AMERICAS"][
            "IDENTIFICACION"
        ].nunique()
        / data["IDENTIFICACION"].nunique(),
        1,
    )
    tiempo_1 = round(data[data["ORDEN"] == 1]["TIEMPO_DESDE_PREGRADO"].mean(), 1)
    tiempo_2 = round(data[data["ORDEN"] == 2]["TIEMPO_DESDE_PREGRADO"].mean(), 1)

    fig = px.bar(
        data[data["ORDEN"] == 1]
        .groupby(["PREGRADO", "POSGRADO"])
        .size()
        .reset_index(name="cuenta"),
        x="cuenta",
        y="POSGRADO",
        color="PREGRADO",
        barmode="stack",
        title="Top Especialidades por Carrera de Pregrado",
    )

    return f"{t_cont}%", f"{t_recompra}%", f"{tiempo_1} años", f"{tiempo_2} años", fig


# === 6. Ejecutar
if __name__ == "__main__":
    app.run(debug=True, port=8052)